In [1]:
import pandas as pd
import sklearn.model_selection as skms
from sklearn.metrics import classification_report
from sklearn.svm import SVC

train = pd.read_csv('Data/train.csv',sep=',')
test = pd.read_csv('Data/test.csv', sep=',')


In [2]:
#df.isnull().sum()

In [5]:
train.tail()

,id,idsito,idmodel,idbrand,lat,lon,ageinmonths,anno,day,ora,...,pcnm9,pcnm10,pcnm11,pcnm12,pcnm13,pcnm14,pcnm15,irr_pvgis_mod,irri_pvgis_mod,kwh
181598,75702,0.15,0.167,0.167,0.438,0.63,0.0,2013,0.477,0.556,...,0.755,0.812,0.424,1.0,0.137,0.405,0.613,0.648,0.158,0.511
181599,75719,0.15,0.167,0.167,0.438,0.63,0.0,2013,0.477,0.556,...,0.755,0.812,0.424,1.0,0.137,0.405,0.613,0.648,0.158,0.511
181600,75736,0.15,0.167,0.167,0.438,0.63,0.0,2013,0.477,0.556,...,0.755,0.812,0.424,1.0,0.137,0.405,0.613,0.648,0.158,0.511
181601,75753,0.15,0.167,0.167,0.438,0.63,0.0,2013,0.477,0.556,...,0.755,0.812,0.424,1.0,0.137,0.405,0.613,0.648,0.158,0.511
181602,75788,0.15,0.167,0.167,0.438,0.63,0.0,2013,0.477,0.556,...,0.755,0.812,0.424,1.0,0.137,0.405,0.613,0.648,0.158,0.511


In [2]:
#train = train.drop("id",1)
#train = train.drop("data",1)
#test = test.drop("id",1)
#test = test.drop("data",1)
from datetime import datetime
#date_object = datetime.strptime(czas, '%m/%d/%Y %H:%M')


In [7]:
from datetime import datetime

def splitDate(x):
    return datetime.strptime(x, '%m/%d/%Y %H:%M').year

train['year']= train['data']

train['data'].apply(splitDate)
print(train['data'].datetime.strptime(x, '%m/%d/%Y %H:%M').year)

AttributeError: 'Series' object has no attribute 'datetime'

In [ ]:
X_train = train.drop("kwh",1)
X_train.head()

In [ ]:
y_train = train.loc[:,'kwh']


In [ ]:
X_test = train.drop("kwh",1)


In [ ]:
y_test = train.loc[:,'kwh']



In [ ]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression().fit(X_train, y_train)
reg.score(X_train, y_train)

In [ ]:
from sklearn.metrics import mean_squared_error
y_pred = np.array(reg.predict(X_test))

In [ ]:
mean_squared_error(y_test, y_pred)

In [ ]:
y_pred[-1]

In [ ]:
y_test.tail(1)